# Weierstrass ℘ Playground — Interactive Trajectory Visualization

This notebook visualizes complex fields derived from the Weierstrass ℘ function on a rectangular lattice and overlays shared second-order trajectories. Choose between:

- **Two-panel mode**: ℘(z) and ℘′(z) with color mapping
- **Three-panel mode**: ℘(z), Re(℘′(z)), and Im(℘′(z)) with grayscale
- **Trajectories**: Follow the second-order ODE z''(t) = -℘(z(t)) z(t)

## Important: How to Use This Notebook

**To avoid errors, please run all cells in order from top to bottom.** 

You can either:
1. Use **Cell → Run All** from the menu, or
2. Run each cell individually using **Shift+Enter**

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Import our Weierstrass library
from weierstrass_lib import *

# Set matplotlib to inline mode
%matplotlib inline

# Configure matplotlib for high quality plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

## Interactive Controls

Configure your visualization parameters below:

In [ ]:
# Global variables for current figure
current_fig = None
output_widget = widgets.Output()

# Visualization mode selection
mode_dropdown = widgets.Dropdown(
    options=[('Two-panel: ℘(z) and ℘′(z)', 'two_panel'),
             ('Three-panel: ℘(z), Re(℘′(z)), Im(℘′(z))', 'three_panel')],
    value='two_panel',
    description='Mode:'
)

# Lattice parameters
p_slider = widgets.FloatSlider(value=11.0, min=1.0, max=20.0, step=0.1, description='p')
q_slider = widgets.FloatSlider(value=5.0, min=1.0, max=20.0, step=0.1, description='q')
N_slider = widgets.IntSlider(value=3, min=0, max=6, description='N (truncation)')

# Rendering parameters
grid_x_slider = widgets.IntSlider(value=100, min=50, max=300, description='Grid X')
grid_y_slider = widgets.IntSlider(value=100, min=50, max=300, description='Grid Y')
contours_slider = widgets.IntSlider(value=10, min=0, max=30, description='# Contours')
vec_density_slider = widgets.IntSlider(value=20, min=0, max=50, description='Vec density')
vec_width_slider = widgets.FloatSlider(value=0.002, min=0.001, max=0.01, step=0.001, description='Vec width')
vec_max_len_slider = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description='Vec max len')
show_vectors_checkbox = widgets.Checkbox(value=True, description='Show vectors')

# Palette parameters
saturation_slider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='Saturation')
value_floor_slider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='Value floor')
mag_scale_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Mag scale')

# Integration parameters
dt_slider = widgets.FloatText(value=0.01, description='dt')
T_slider = widgets.FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='T (duration)')
blowup_thresh_slider = widgets.FloatSlider(value=10.0, min=1.0, max=50.0, step=1.0, description='Blow-up |Δz|')
emoji_size_slider = widgets.IntSlider(value=20, min=10, max=50, description='Emoji size')

# Particle list management
particle_list = []
particles_container = widgets.VBox()

def create_particle_row(idx=0, z0_default='5.5+0j', v0_default='0+1j'):
    """Create a particle input row."""
    z0_text = widgets.Text(value=z0_default, description=f'z0 #{idx}')
    v0_text = widgets.Text(value=v0_default, description=f'v0 #{idx}')
    remove_btn = widgets.Button(description='Remove', button_style='danger', layout=widgets.Layout(width='80px'))
    
    def remove_particle(b):
        if len(particle_list) > 1:
            particle_list.remove((z0_text, v0_text, remove_btn, row))
            update_particles_display()
    
    remove_btn.on_click(remove_particle)
    row = widgets.HBox([z0_text, v0_text, remove_btn])
    
    return z0_text, v0_text, remove_btn, row

def add_particle(b=None, z0_default='5.5+0j', v0_default='0+1j'):
    """Add a new particle."""
    idx = len(particle_list)
    particle_row = create_particle_row(idx, z0_default, v0_default)
    particle_list.append(particle_row)
    update_particles_display()

def update_particles_display():
    """Update the particles display."""
    for i, (z0_text, v0_text, remove_btn, row) in enumerate(particle_list):
        z0_text.description = f'z0 #{i}'
        v0_text.description = f'v0 #{i}'
    
    particles_container.children = [row for _, _, _, row in particle_list]

def get_particles():
    """Get current particle initial conditions."""
    particles = []
    for z0_text, v0_text, _, _ in particle_list:
        try:
            z0 = complex(z0_text.value)
            v0 = complex(v0_text.value)
            particles.append((z0, v0))
        except ValueError:
            continue
    return particles

# Initialize with default particles
add_particle(z0_default='5.5+0j', v0_default='0+1j')  # z = 5.5, z' = i
add_particle(z0_default='5+0j', v0_default='0+1j')    # z = 5, z' = i
add_particle(z0_default='7+0j', v0_default='0+0j')    # z = 7

add_particle_btn = widgets.Button(description='Add Particle', button_style='success')
add_particle_btn.on_click(add_particle)

# Control buttons
render_btn = widgets.Button(description='Render', button_style='primary')
save_btn = widgets.Button(description='Save PNG', button_style='info')

# Create the UI layout
mode_box = widgets.VBox([
    widgets.HTML("<h3>Visualization Mode</h3>"),
    mode_dropdown
])

lattice_box = widgets.VBox([
    widgets.HTML("<h3>Lattice Parameters</h3>"),
    p_slider, q_slider, N_slider
])

rendering_box = widgets.VBox([
    widgets.HTML("<h3>Rendering</h3>"),
    grid_x_slider, grid_y_slider, contours_slider,
    show_vectors_checkbox, vec_density_slider, vec_width_slider, vec_max_len_slider
])

palette_box = widgets.VBox([
    widgets.HTML("<h3>Palette</h3>"),
    saturation_slider, value_floor_slider, mag_scale_slider
])

integration_box = widgets.VBox([
    widgets.HTML("<h3>Integration</h3>"),
    dt_slider, T_slider, blowup_thresh_slider, emoji_size_slider
])

particles_box = widgets.VBox([
    widgets.HTML("<h3>Particles</h3>"),
    particles_container,
    add_particle_btn
])

controls_box = widgets.VBox([
    widgets.HTML("<h3>Controls</h3>"),
    render_btn, save_btn
])

# Layout in three columns
left_column = widgets.VBox([mode_box, lattice_box, rendering_box])
middle_column = widgets.VBox([palette_box, integration_box])
right_column = widgets.VBox([particles_box, controls_box])

ui = widgets.HBox([left_column, middle_column, right_column])

# Display the interface at the top
display(ui)

print("Weierstrass ℘ Playground loaded!")
print("Configure parameters above and click 'Render' to generate visualization.")
print("\nTips:")
print("- Try p=11, q=5, N=3 for a good starting point")
print("- Two-panel mode shows ℘(z) and ℘′(z) with color mapping")
print("- Three-panel mode shows ℘(z), Re(℘′(z)), and Im(℘′(z)) in grayscale")
print("- Higher N values give more accurate ℘ function but slower computation")
print("- Trajectories follow z''(t) = -℘(z(t)) * z(t)")
print("- 💥 marks indicate trajectory blow-ups near poles")

## Mathematical Background

The mathematical functions are implemented in the `weierstrass_lib.py` module, including:

- **Core functions**: `wp_rect()`, `wp_deriv()`, `wp_and_deriv()`
- **Field sampling**: `field_grid()` with pole detection
- **Visualization**: `soft_background()`, `grayscale_background()`, contours, vectors
- **Integration**: `integrate_second_order_with_blowup()` with RK4 and blow-up detection
- **Trajectory handling**: `wrap_with_breaks()`, `wrap_point()` for fundamental cell wrapping

## Main Rendering Function

In [ ]:
def render_playground():
    """Main rendering function for the Weierstrass playground."""
    global current_fig
    
    with output_widget:
        clear_output(wait=True)
        
        # Get parameters
        mode = mode_dropdown.value
        p, q, N = p_slider.value, q_slider.value, N_slider.value
        nx, ny = grid_x_slider.value, grid_y_slider.value
        n_contours = contours_slider.value
        vec_density = vec_density_slider.value if show_vectors_checkbox.value else 0
        vec_width = vec_width_slider.value
        vec_max_len = vec_max_len_slider.value
        
        saturation = saturation_slider.value
        value_floor = value_floor_slider.value
        mag_scale = mag_scale_slider.value
        
        dt = dt_slider.value
        T = T_slider.value
        blow_thresh = blowup_thresh_slider.value
        emoji_size = emoji_size_slider.value
        
        particles = get_particles()
        
        print(f"Rendering {mode} with p={p}, q={q}, N={N}, particles={len(particles)}")
        print(f"Grid: {nx}×{ny}, dt={dt}, T={T}")
        
        # Create figure based on mode
        if mode == 'two_panel':
            fig, axes = create_two_panel_figure(p, q)
            ax1, ax2 = axes
            
            # Compute fields
            X1, Y1, F1, M1 = field_grid(p, q, 'wp', N, nx, ny)
            X2, Y2, F2, M2 = field_grid(p, q, 'wp_deriv', N, nx, ny)
            
            # Create backgrounds
            bg1 = soft_background(F1, M1, saturation, mag_scale, value_floor)
            bg2 = soft_background(F2, M2, saturation, mag_scale, value_floor)
            
            # Display backgrounds
            ax1.imshow(bg1, extent=[0, p, 0, q], origin='lower', aspect='equal')
            ax2.imshow(bg2, extent=[0, p, 0, q], origin='lower', aspect='equal')
            
            # Add contours
            add_topo_contours(ax1, X1, Y1, F1, M1, n_contours)
            add_topo_contours(ax2, X2, Y2, F2, M2, n_contours)
            
            # Add vector fields
            if vec_density > 0:
                vector_overlay(ax1, X1, Y1, F1, M1, vec_density, vec_width, vec_max_len)
                vector_overlay(ax2, X2, Y2, F2, M2, vec_density, vec_width, vec_max_len)
        
        else:  # three_panel mode
            fig, axes = create_three_panel_figure(p, q)
            ax1, ax2, ax3 = axes
            
            # Compute fields
            X1, Y1, F1, M1 = field_grid(p, q, 'wp', N, nx, ny)
            X2, Y2, F2, M2 = field_grid(p, q, 'wp_deriv', N, nx, ny)
            
            # Create backgrounds
            bg1 = soft_background(F1, M1, saturation, mag_scale, value_floor)
            bg2 = grayscale_background(np.real(F2), M2, value_floor)
            bg3 = grayscale_background(np.imag(F2), M2, value_floor)
            
            # Display backgrounds
            ax1.imshow(bg1, extent=[0, p, 0, q], origin='lower', aspect='equal')
            ax2.imshow(bg2, extent=[0, p, 0, q], origin='lower', aspect='equal', cmap='gray')
            ax3.imshow(bg3, extent=[0, p, 0, q], origin='lower', aspect='equal', cmap='gray')
            
            # Add contours
            add_topo_contours(ax1, X1, Y1, F1, M1, n_contours)
            # For grayscale panels, add contours of the real/imaginary parts
            if n_contours > 0:
                re_F2 = np.real(F2)
                im_F2 = np.imag(F2)
                re_F2 = np.where(M2, re_F2, np.nan)
                im_F2 = np.where(M2, im_F2, np.nan)
                
                if not np.all(np.isnan(re_F2)):
                    vmin, vmax = np.nanmin(re_F2), np.nanmax(re_F2)
                    if vmax > vmin:
                        levels = np.linspace(vmin, vmax, n_contours)
                        ax2.contour(X2, Y2, re_F2, levels=levels, colors='black', alpha=0.3, linewidths=0.5)
                
                if not np.all(np.isnan(im_F2)):
                    vmin, vmax = np.nanmin(im_F2), np.nanmax(im_F2)
                    if vmax > vmin:
                        levels = np.linspace(vmin, vmax, n_contours)
                        ax3.contour(X2, Y2, im_F2, levels=levels, colors='black', alpha=0.3, linewidths=0.5)
            
            # Add vector fields only to the first panel in three-panel mode
            if vec_density > 0:
                vector_overlay(ax1, X1, Y1, F1, M1, vec_density, vec_width, vec_max_len)
        
        # Integrate and plot trajectories
        trajectories = []
        colors = plt.cm.tab10(np.linspace(0, 1, len(particles)))
        
        for i, (z0, v0) in enumerate(particles):
            try:
                trajectory, blowup_point = integrate_second_order_with_blowup(
                    z0, v0, dt, T, p, q, N, blow_thresh
                )
                trajectories.append((trajectory, blowup_point))
            except Exception as e:
                print(f"Error integrating particle {i}: {e}")
                trajectories.append((np.array([z0]), None))
        
        # Plot trajectories on all axes
        plot_trajectories_on_axes(axes, trajectories, colors, p, q, emoji_size)
        
        plt.tight_layout()
        current_fig = fig
        plt.show()

def save_figure(b=None):
    """Save current figure as PNG."""
    global current_fig
    if current_fig is not None:
        filename = 'weierstrass_playground.png'
        current_fig.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Figure saved as {filename}")
    else:
        print("No figure to save. Please render first.")

# Connect button callbacks
render_btn.on_click(lambda b: render_playground())
save_btn.on_click(save_figure)

## Interactive Interface

## Output Area

Visualization results will appear below:

In [ ]:
# Display the output widget for visualizations
display(output_widget)